# **Image classification**

## 1. Install necessary libraries

In [ ]:
%pip install numpy
%pip install tensorflow
%pip install matplotlib

## 2. Load images

In [ ]:
import tensorflow as tf

# Set the path to the dataset
dataset = tf.keras.utils.image_dataset_from_directory("Dataset")

In [ ]:
# Get class labels from the directory names
labels_name = dataset.class_names
labels_name

## 3. Prepare the images

In [ ]:
import numpy as np

# Extract images and labels from the dataset
images = []
labels = []

for image, label in dataset:
    images.extend(image.numpy())
    labels.extend(label.numpy())

# Convert the lists to NumPy arrays
images = np.array(images)
labels = np.array(labels)

# Check the shape of the arrays
print("Images shape:", images.shape)
print("Labels shape:", labels.shape)

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Print the shapes of the resulting sets
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)


In [ ]:
# Normalize the pixel values to the range [0, 1]
X_train = X_train / 255.0
X_test = X_test / 255.0

In [ ]:
# Resize images to a 32x32 size
image_size = (32, 32)

X_train = tf.image.resize(X_train, image_size)
X_test = tf.image.resize(X_test, image_size)

In [ ]:
import matplotlib.pyplot as plt

# View few images 
for i in range(9):
	plt.subplot(330 + 1 + i)
	plt.imshow(X_train[i])
plt.show()

In [ ]:
from keras.utils import to_categorical

# One-hot encode the target labels
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
     
# Print 4 examples to check the one-hot encoding
print(y_train[0:4])

## 4. Build the model

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

model = Sequential()

model.add(Conv2D(32, (3, 3), activation="relu", input_shape=(32, 32, 3)))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation="relu"))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.2))

model.add(Dense(6, activation="softmax"))


## 5. Compile the model

In [ ]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
print(model.summary())

## 6. Train the model

In [ ]:
model.fit(X_train, y_train, 
          epochs=25, 
          batch_size=64, 
          validation_data=(X_test, y_test), 
          verbose=1)


## 7. Evaluate the model on the test set

In [ ]:
loss, acc = model.evaluate(X_test, y_test)
print("The accuracy of the model is", round(acc*100, 2), "%.")

## 8. Make a random prediction

In [ ]:
import random

random_number = random.randint(0, len(X_test))
test_image = X_test[random_number]
test_input=np.expand_dims(test_image, 0)
actual= np.argmax(y_test[random_number], axis=None)

y_pred = model.predict(test_input)
prediction = np.argmax(y_pred, axis=None)

# Visualize the test image and display the original and predicted class labels
plt.figure(figsize=(2, 2))
plt.imshow(test_image)

original_label=labels_name[actual]
prediction_label=labels_name[prediction]

print("The original class is:", original_label)
print("The predicted class is:", prediction_label)

